<a href="https://colab.research.google.com/github/Lionel98/CTI/blob/main/Test_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.1 MB/s eta 0:00:00


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.9 MB/s eta 0:00:00


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import pandas as pd
from transformers import pipeline

def roberta_test(csv_file):
    # Load the pipeline
    pipe = pipeline(model="ltkw98/roberta-large-mnli_Fintuned_Fold3")

    # Read the existing CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Iterate over the sentences and update the columns
    for i in range(2358):
        # Update the "Model's Response" column
        result = pipe(df['sentence'][i], top_k=1, truncation=True)
        predicted_label = result[0]['label']
        df.loc[i, "Model's Response"] = predicted_label

        # Update the "Log Probability" column
        result = pipe(df['sentence'][i], top_k=5, truncation=True)
        log_probs = [str(entry) for entry in result]
        df.loc[i, "Log Probability"] = ', '.join(log_probs)

    # Save the DataFrame to a new CSV file
    new_csv_file = "roberta_result_set.csv"
    df.to_csv(new_csv_file, index=False)

In [13]:
import pandas as pd
from transformers import pipeline

def bert_test(csv_file):
    # Load the pipeline
    pipe = pipeline(model="ltkw98/Bert4")

    # Read the existing CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Iterate over the sentences and update the columns
    for i in range(2358):
        # Update the "Model's Response" column
        result = pipe(df['sentence'][i], top_k=1, truncation=True)
        predicted_label = result[0]['label']
        df.loc[i, "Model's Response"] = predicted_label

        # Update the "Log Probability" column
        result = pipe(df['sentence'][i], top_k=5, truncation=True)
        log_probs = [str(entry) for entry in result]
        df.loc[i, "Log Probability"] = ', '.join(log_probs)

    # Save the DataFrame to a new CSV file
    new_csv_file = "bert_result_set.csv"
    df.to_csv(new_csv_file, index=False)

In [7]:
from datasets import load_dataset

def model_response(csv_file):
    # Load the dataset
    test = load_dataset("csv", data_files=csv_file)

    total_rows = len(test['train']["Model's Response"])
    matching_count = 0

    for i in range(total_rows):
        if test['train']["Model's Response"][i] == test['train']['tec_name'][i]:
            matching_count += 1

    matching_percentage = (matching_count / total_rows) * 100

    print(f"Number of Sentences Matched: {matching_count}")
    print(f"Matching Percentage: {matching_percentage}%")

    return matching_count, matching_percentage

In [8]:
from datasets import load_dataset
import re

def Top5_Response(csv_file):
    # Load the dataset
    test = load_dataset("csv", data_files=csv_file)

    total_rows = len(test['train']['tec_name'])
    match_count = 0

    for i in range(total_rows):
        tec_name = test['train']['tec_name'][i]
        log_probabilities_str = test['train']['Log Probability'][i]
        log_probabilities = re.findall(r"'label': '([^']+)'", log_probabilities_str)

        for label in log_probabilities:
            if tec_name.lower() in label.lower():
                match_count += 1
                break  # Break out of the inner loop once a match is found

    match_percentage = (match_count / total_rows) * 100

    print(f"Number of Matches: {match_count}")
    print(f"Matching Percentage: {match_percentage}%")

    return match_count, match_percentage

In [5]:
csv_file = "test_result.csv"
roberta_test(csv_file)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [14]:
csv_file = "test_result.csv"
bert_test(csv_file)

In [9]:
csv_file = "/content/roberta_result_set.csv"
matching_count, matching_percentage = model_response(csv_file)

  0%|          | 0/1 [00:00<?, ?it/s]

Number of Sentences Matched: 1859
Matching Percentage: 78.83799830364715%


In [10]:
csv_file = "/content/roberta_result_set.csv"
matching_count, matching_percentage = Top5_Response(csv_file)

  0%|          | 0/1 [00:00<?, ?it/s]

Number of Matches: 2167
Matching Percentage: 91.89991518235793%


In [15]:
csv_file = "/content/bert_result_set.csv"
matching_count, matching_percentage = model_response(csv_file)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c21394ba5935cfc0/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Number of Sentences Matched: 1811
Matching Percentage: 76.80237489397794%


In [16]:
csv_file = "/content/bert_result_set.csv"
matching_count, matching_percentage = Top5_Response(csv_file)

  0%|          | 0/1 [00:00<?, ?it/s]

Number of Matches: 2128
Matching Percentage: 90.2459711620017%
